In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
import keras 
from keras_preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.optimizers import Adam
import pickle

In [2]:
# loding dataset and splitting into train and test set

(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [3]:
# splitting training data into training and validation

x_train,x_validation,y_train,y_validation = train_test_split(x_train,y_train,test_size=0.10,random_state=111)
print(x_train.shape,y_train.shape)
print(x_validation.shape,y_validation.shape)

(54000, 28, 28) (54000,)
(6000, 28, 28) (6000,)


In [4]:
# conversion to 4 dimension
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_validation = x_validation.reshape(x_validation.shape[0],28,28,1)

input_shape = (28, 28, 1)

print(x_train.shape)
print(x_test.shape)
print(x_validation.shape)

(54000, 28, 28, 1)
(10000, 28, 28, 1)
(6000, 28, 28, 1)


In [5]:
# Integer to float conversion
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_validation = x_validation.astype('float32')

In [6]:
# Normalization : rescale data between 0 and 1 ;s
x_train /= 255
x_test /= 255
x_validation /= 255

In [7]:
x_train = x_train / 255
x_test = x_test / 255
x_validation =x_validation /255

In [8]:
# one hot encoding
y_train =to_categorical(y_train,10)
y_test = to_categorical(y_test,10)
y_validation =to_categorical(y_validation,10)

In [9]:
def Mymodel():
  nooffilters=60
  sizeoffilter1=(5,5)
  sizeoffilter2=(3,3)
  sizeofpool=(2,2)
  noofnode =500

  model = Sequential()
  model.add((Conv2D(nooffilters,sizeoffilter1,input_shape=input_shape,activation='relu')))
  model.add((Conv2D(nooffilters,sizeoffilter1,activation='relu')))
  model.add(MaxPooling2D(pool_size=sizeofpool))
  model.add((Conv2D(nooffilters//2,sizeoffilter2,activation='relu')))
  model.add((Conv2D(nooffilters//2,sizeoffilter2,activation='relu')))
  model.add(MaxPooling2D(pool_size=sizeofpool))
  model.add(Dropout(0.5))

  model.add(Flatten())
  model.add(Dense(noofnode,activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(10,activation='softmax'))
  model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
  return model

In [10]:
model = Mymodel()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 60)        1560      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 60)        90060     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 60)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 30)          16230     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 6, 30)          8130      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3, 3, 30)          0         
_________________________________________________________________
dropout (Dropout)            (None, 3, 3, 30)          0

In [11]:
# construct the training image generator for data augmentation
datagen = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [12]:
datagen.fit(x_train)

In [13]:
# history = model.fit_generator(datagen.flow(x_train,y_train,
#                                  batch_size = 50),
#                                  steps_per_epoch = 2000,
#                                  epochs = 1,
#                                  validation_data =(x_validation,y_validation))

In [ ]:
history = model.fit(x_train,y_train,batch_size=50,steps_per_epoch=2000,epochs=18,validation_data=(x_validation,y_validation))

Epoch 1/18
 504/2000 [======>.......................] - ETA: 7:18 - loss: 2.3021 - accuracy: 0.1097

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training','validation'])
plt.title('loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['training','validation'])
plt.title('accuracy')
plt.xlabel('epoch')
plt.show()

In [ ]:
score = model.evaluate(x_test,y_test,verbose=0)
print('Test score = ',score[0])
print('Test Accuracy = ',score[1])

In [ ]:
model.save('digit_classifier.model')